In [1]:
import veloxchem as vlx
import adcc
from mpi4py import MPI
import sys
import numpy
from scipy.sparse import linalg

numpy.set_printoptions(precision=7, suppress=True)

* Warning * Environment variable OMP_NUM_THREADS not set.
* Warning * Setting OMP_NUM_THREADS to 8.


### Prepare a VeloxChem SCF calculation

In [2]:
# Input settings
molecule_string = """
    O 0 0 0
    H 0 0 1.795239827225189
    H 1.693194615993441 0 -0.599043184453037"""

#molecule_string = """
#    H 0 0 0
#    H 0 0 1.795239827225189
#    """

basis_set_label = 'STO-3G'
scf_settings = {'conv_thresh': 1.0e-6}
method_settings = {} #{'xcfun': 'b3lyp', 'grid_level': 4}
rsp_settings = {'conv_thresh': 1.0e-4, 'nstates': 3}

In [3]:
# Communicator and output stream
comm = MPI.COMM_WORLD
ostream = vlx.OutputStream(sys.stdout)

In [4]:
# Molecule and basis set
molecule = vlx.Molecule.read_str(molecule_string, units='au')
basis = vlx.MolecularBasis.read(molecule, basis_set_label)

ostream.print_block(molecule.get_string())
ostream.print_block(basis.get_string('Atomic Basis', molecule))
ostream.flush()

                                              Molecular Geometry (Angstroms)                                              
                                                                                                                          
                          Atom         Coordinate X          Coordinate Y          Coordinate Z                           
                                                                                                                          
                           O           0.000000000000        0.000000000000        0.000000000000                         
                           H           0.000000000000        0.000000000000        0.950000004704                         
                           H           0.896000004436        0.000000000000       -0.317000001569                         
                                                                                                                          
                

In [5]:
# MockTask class needed to run adcc with vlx
class MockTask:
    def __init__(self, mol, basis, comm, ostream):

        self.molecule = mol
        self.ao_basis = basis
        self.mpi_comm = comm
        self.ostream = ostream

In [6]:
# SCF
task = MockTask(molecule, basis, comm, ostream)
scfdrv = vlx.ScfRestrictedDriver(comm, ostream)
scfdrv.update_settings(scf_settings, method_settings)
scfdrv.compute(molecule, basis)
scfdrv.task = task

                                                                                                                          
                                            Self Consistent Field Driver Setup                                            
                                                                                                                          
                   Wave Function Model             : Spin-Restricted Hartree-Fock                                         
                   Initial Guess Model             : Superposition of Atomic Densities                                    
                   Convergence Accelerator         : Two Level Direct Inversion of Iterative Subspace                     
                   Max. Number of Iterations       : 50                                                                   
                   Max. Number of Error Vectors    : 10                                                                   
                

In [7]:
# SCF first-order properties
scf_prop = vlx.ScfFirstOrderProperties(comm, ostream)
scf_prop.compute(molecule, basis, scfdrv.scf_tensors)
scf_prop.print_properties(molecule)

                                                                                                                          
                                                Ground-State Dipole Moment                                                
                                               ----------------------------                                               
                                                                                                                          
                                   X   :         0.546314 a.u.         1.388593 Debye                                     
                                   Y   :         0.000000 a.u.         0.000000 Debye                                     
                                   Z   :         0.386302 a.u.         0.981881 Debye                                     
                                 Total :         0.669095 a.u.         1.700671 Debye                                     
                

### Excited-State Calculation via CIS (TDHF/TDA)

In [8]:
# TDHF/TDA, i.e. CIS
tda_drv = vlx.TDAExciDriver(comm, ostream)
tda_drv.update_settings(rsp_settings, method_settings)
tda_results = tda_drv.compute(molecule, basis, scfdrv.scf_tensors)
tda_drv.ostream.flush()

                                                                                                                          
                                                     TDA Driver Setup                                                     
                                                                                                                          
                               Number of States                : 3                                                        
                               Max. Number of Iterations       : 150                                                      
                               Convergence Threshold           : 1.0e-04                                                  
                               ERI Screening Scheme            : Cauchy Schwarz + Density                                 
                               ERI Screening Threshold         : 1.0e-15                                                  
                

In [9]:
# RPA = TDHF
rpa_drv = vlx.LinearResponseEigenSolver(comm, ostream)
rpa_drv.update_settings(rsp_settings, method_settings)
rpa_results = rpa_drv.compute(molecule, basis, scfdrv.scf_tensors)
rpa_drv.ostream.flush()

                                                                                                                          
                                            Linear Response EigenSolver Setup                                             
                                                                                                                          
                               Number of States                : 3                                                        
                               Max. Number of Iterations       : 150                                                      
                               Convergence Threshold           : 1.0e-04                                                  
                               ERI Screening Scheme            : Cauchy Schwarz + Density                                 
                               ERI Screening Threshold         : 1.0e-15                                                  
                

               *** Linear response converged in 4 iterations. Time: 0.12 sec                                              
                                                                                                                          
                                                                                                                          


In [10]:
# Get MO coefficients

nocc = molecule.number_of_alpha_electrons()
mo = scfdrv.scf_tensors['C'] # MO coefficients
mo_occ = mo[:, :nocc]        # occupied
mo_vir = mo[:, nocc:]        # virtual

nocc = mo_occ.shape[1]
nvir = mo_vir.shape[1]

#eig_vec = eig_vecs[:, 0].copy() # first eigenvector
#print(eig_vec)
#print("mo_occ:")
#print(mo_occ)
#print()
#print("mo_vir:")
#print(mo_vir)

In [11]:
# Get the overlap and Fock matrices

print(scfdrv.scf_tensors.keys())
fock = scfdrv.scf_tensors['F'][0]
ovlp = scfdrv.scf_tensors['S']
# Transform them from AO to MO basis
print(numpy.matmul(mo.T, numpy.matmul(ovlp, mo)))
print(numpy.matmul(mo.T, numpy.matmul(fock, mo)))

dict_keys(['C', 'E', 'S', 'D', 'F'])
[[ 1.  0. -0. -0.  0. -0. -0.]
 [ 0.  1. -0.  0.  0. -0. -0.]
 [-0. -0.  1. -0. -0. -0.  0.]
 [-0.  0. -0.  1.  0. -0.  0.]
 [ 0.  0. -0.  0.  1.  0.  0.]
 [-0. -0. -0. -0.  0.  1. -0.]
 [-0. -0.  0.  0.  0. -0.  1.]]
[[-20.2333974  -0.         -0.         -0.         -0.         -0.
   -0.       ]
 [ -0.         -1.2657146  -0.         -0.         -0.          0.0000002
    0.       ]
 [ -0.         -0.         -0.6292671  -0.         -0.         -0.
    0.0000001]
 [ -0.         -0.         -0.         -0.441668   -0.          0.
    0.       ]
 [ -0.         -0.         -0.         -0.         -0.3876451  -0.
    0.       ]
 [ -0.          0.0000002  -0.          0.         -0.          0.6028394
    0.       ]
 [ -0.          0.          0.0000001   0.          0.          0.
    0.7659184]]


In [12]:
# Get the CIS eigenvalues and eigenvectors

tda_eig_vals = tda_results["eigenvalues"]
tda_eig_vecs = tda_results["eigenvectors"]
tda_size = tda_eig_vecs[:,0].shape
tda_eig_vec=tda_eig_vecs[:,0].copy()
tda_eig_vec_as_mat = tda_eig_vec.reshape(nocc, nvir) #/ numpy.sqrt(2.0)

print(numpy.linalg.norm(tda_eig_vec))

0.9999999999999999


The RPA eigenvectors are usually normalized according to
\begin{equation}
    \mathbf{X}^\dagger \mathbf{X} - \mathbf{Y}^\dagger \mathbf{Y} = 1
\end{equation}
where $\mathbf{X}$ is the excitation part and $\mathbf{Y}$ the de-excitation part.

In [15]:
# Get the RPA eigenvalues and eigenvectors
#rpa_results.keys()
rpa_eig_vals = rpa_results["eigenvalues"]
rpa_eig_vecs = rpa_results["eigenvectors"]
#print(rpa_eig_vecs)

# Get the excitation and de-excitation part of the first excited state
rpa_vec1 = rpa_eig_vecs[:, 0].copy()
rpa_exc_vec = rpa_eig_vecs[:nocc*nvir, 0].copy()
rpa_deexc_vec = rpa_eig_vecs[nocc*nvir:, 0].copy()
print(rpa_eig_vecs)
print()
print(rpa_exc_vec)
print()
print(rpa_deexc_vec)

print("\nexc_vec")
print(rpa_exc_vec.reshape(nocc, nvir))
print("\ndeexc_vec")
print(rpa_deexc_vec.reshape(nocc, nvir))
print()
print(rpa_drv.lrvec2mat(rpa_vec1, nocc, nocc+nvir))

#eig_vecs_norm = numpy.matmul(rpa_exc_vec.T, rpa_exc_vec) - numpy.matmul(rpa_deexc_vec.T, rpa_deexc_vec)
eig_vecs_norm = numpy.linalg.norm(rpa_exc_vec)**2 - numpy.linalg.norm(rpa_deexc_vec)**2
print("\nNorm of vectors:")
print(eig_vecs_norm)

# Construct the plus/minus combinations and reshape to matrix form
xpy = (rpa_exc_vec + rpa_deexc_vec).reshape(nocc, nvir)
xmy = (rpa_exc_vec - rpa_deexc_vec).reshape(nocc, nvir)
print("\nxpy:")
print(xpy)
print("\nxmy:")
print(xmy)

[[ 0.        -0.         0.0005798]
 [-0.         0.        -0.0000008]
 [ 0.         0.        -0.0367831]
 [ 0.         0.        -0.0000695]
 [-0.         0.        -0.0005901]
 [ 0.        -0.         0.1259496]
 [ 0.        -0.         0.6957657]
 [-0.         0.        -0.000108 ]
 [-0.7076842  0.000658   0.       ]
 [ 0.0006569  0.7072696  0.       ]
 [ 0.        -0.         0.0001585]
 [ 0.        -0.         0.0000009]
 [ 0.        -0.         0.0103634]
 [ 0.        -0.         0.0000262]
 [-0.         0.         0.0000412]
 [-0.         0.        -0.0274986]
 [ 0.        -0.         0.0210533]
 [-0.         0.        -0.0000381]
 [-0.0285899  0.0000386  0.       ]
 [ 0.000002   0.0151911 -0.       ]]

[ 0.        -0.         0.         0.        -0.         0.
  0.        -0.        -0.7076842  0.0006569]

[ 0.         0.         0.         0.        -0.        -0.
  0.        -0.        -0.0285899  0.000002 ]

exc_vec
[[ 0.        -0.       ]
 [ 0.         0.       ]
 [-0. 

### Perform an analogous calculation with $\texttt{adcc}$ for comparison

In [ ]:
adc1 = adcc.run_adc(scfdrv, method='adc1', n_singlets=3, conv_tol=1e-4)

In [ ]:
adc1_eig_vec = adc1.excitation_vector[0]["ph"].to_ndarray()
adc1_eig_vals = adc1.excitation_energy
adc1.state_dipole_moment

In [ ]:
# Compare the eigenvalues and eigenvectors
print("vlx eigenvalues:")
print(tda_eig_vals)
print()
print("adcc eigenvalues:")
print(adc1_eig_vals)
print()
print("vlx eigenvector:")
print(tda_eig_vec_as_mat)
print()
print("adcc eigenvector:")
print(adc1_eig_vec)
print()
print("vlx eigenvector divided by square root 2:")
print(tda_eig_vec_as_mat/numpy.sqrt(2))

### Calculate one- and two-particle density matrices

\begin{align}
    \gamma_{ij} &= - \sum_{a} x_{ia} x_{ja} \\
    \gamma_{ab} &= + \sum_{i} x_{ia} x_{ib} \\
    \Gamma_{iajb} &= - x_{ib} x_{ja}
\end{align}

In [ ]:
# VeloxChem unrelaxed one-particle density matrix and two-particle density matrix

vlx_dm_oo = -numpy.einsum('ia,ja->ij', tda_eig_vec_as_mat, tda_eig_vec_as_mat)
vlx_dm_vv = numpy.einsum('ia,ib->ab', tda_eig_vec_as_mat, tda_eig_vec_as_mat)
vlx_DM_ovov = -numpy.einsum('ib,ja->iajb', tda_eig_vec_as_mat, tda_eig_vec_as_mat)

print(vlx_dm_oo)

In [ ]:
# adcc density matrices

adcc_dm_oo = -numpy.einsum('ia,ja->ij', adc1_eig_vec, adc1_eig_vec)
adcc_dm_vv = numpy.einsum('ia,ib->ab', adc1_eig_vec, adc1_eig_vec)
adcc_DM_ovov = -numpy.einsum('ib,ja->iajb', adc1_eig_vec, adc1_eig_vec)

#print(adcc_dm_oo)

# adcc Fock matrix
foo = adc1.reference_state.fock("o1o1").to_ndarray()
fvv = adc1.reference_state.fock("v1v1").to_ndarray()
ovov = adc1.reference_state.eri("o1v1o1v1").to_ndarray()

In the case of RPA, the unrelaxed one-particle density matrices are given by

\begin{align}
    \gamma_{ij} &= -\frac12 \sum_{a} \big[ (x_{ia} + y_{ia})(x_{ja} + y_{ja})
        + (x_{ia} - y_{ia})(x_{ja} - y_{ja}) \big] \\
    \gamma_{ab} &= +\frac12 \sum_{i} \big[ (x_{ia} + y_{ia})(x_{ib} + y_{ib})
        + (x_{ia} - y_{ia})(x_{ib} - y_{ib}) \big] \\
\end{align}

where $x_{ia}$ is the excitation part of the eigenvectors and $y_{ia}$ the de-excitation part.

The two-particle density matrix could be given by something like this:
\begin{equation}
    \Gamma_{iajb} = - \frac12 \big[ (x_{ib} + y_{ib})(x_{ja} + y_{ja}) + (x_{ib} - y_{ib})(x_{ja} - y_{ja}) \big]
\end{equation}
$\color{red}{Check~normalization!!}$

In [ ]:
# RPA unrelaxed one-particle density matrix
rpa_dm_oo = -1.0*(numpy.einsum('ia,ja->ij', xpy, xpy)
             + numpy.einsum('ia,ja->ij', xmy, xmy))
rpa_dm_vv = 1.0*(numpy.einsum('ia,ib->ab', xpy, xpy)
             + numpy.einsum('ia,ib->ab', xmy, xmy))

# RPA two-particle density matrix
rpa_DM_ovov = -1.0*(numpy.einsum('ib,ja->iajb', xpy, xpy)
                   + numpy.einsum('ib,ja->iajb', xmy, xmy))

print(rpa_dm_oo)
print()
print(rpa_dm_vv)

The energy can always be expressed as
\begin{equation}
    E = \sum_{pq} \gamma_{pq} \, f_{pq} + \sum_{pqrs} \Gamma_{pqrs} \langle pq || rs \rangle
\end{equation}

In [ ]:
# Confirm the density matrices by calculating the excitation energy
# as a contraction of the densities with the Fock matrix and ERI tensor

adcc_excitation_energy = (numpy.einsum('ij,ij->',adcc_dm_oo,foo)
                     + numpy.einsum('ab,ab->', adcc_dm_vv, fvv)
                     + numpy.einsum('iajb,iajb->', adcc_DM_ovov, ovov)
                    )
print("adcc excitation energy:", adcc_excitation_energy)

In [ ]:
# Analogous confirmation of the vlx density matrices
# two different spin blocks of the antisymmmetrized ERIs need to be considered:
# <aa||aa> = <bb||bb> as well as <ab||ba> = <ba||ab>
vlx_excitation_energy = (1.0*numpy.einsum('ij,ij->', vlx_dm_oo,foo[:nocc,:nocc])
                        +1.0*numpy.einsum('ab,ab->', vlx_dm_vv, fvv[:nvir,:nvir])
                        +1.0*numpy.einsum('iajb,iajb->', vlx_DM_ovov, ovov[:nocc, :nvir, :nocc, :nvir])##aaaa
                        +1.0*numpy.einsum('iajb,iajb->', vlx_DM_ovov, ovov[:nocc, nvir:, nocc:, :nvir])##abba
                        )
print(" vlx excitation energy:", vlx_excitation_energy)

In [ ]:
# Re-confirm the adcc excitation energy by considering only the spin blocks vlx takes
adcc_excitation_energy = (1.0*numpy.einsum('ij,ij->', vlx_dm_oo,foo[:nocc,:nocc])
                        +1.0*numpy.einsum('ab,ab->', vlx_dm_vv, fvv[:nvir,:nvir])
                        +2.0*numpy.einsum('iajb,iajb->', adcc_DM_ovov[:nocc, :nvir, :nocc, :nvir], ovov[:nocc, :nvir, :nocc, :nvir])
                        +2.0*numpy.einsum('iajb,iajb->', adcc_DM_ovov[:nocc, :nvir, :nocc, :nvir], ovov[:nocc, nvir:, nocc:, :nvir])
    )
print("adcc excitation energy:", adcc_excitation_energy)

In [ ]:
# Take a look at the different shapes of the integrals and 2PDMs
print("ovov.shape:        ", ovov.shape)
print("adcc_DM_ovov.shape:", adcc_DM_ovov.shape)
print("vlx_DM_ovov.shape: ", vlx_DM_ovov.shape)

In [ ]:
# Confirm the RPA density matrices by calculating the excitation energy with them
print("RPA first excitation energy:\n", rpa_eig_vals[0])

rpa_excitation_energy = (1.0*numpy.einsum('ij,ij->', rpa_dm_oo, foo[:nocc,:nocc])
                        +1.0*numpy.einsum('ab,ab->', rpa_dm_vv, fvv[:nvir,:nvir])
                        +1.0*numpy.einsum('iajb,iajb->', rpa_DM_ovov, ovov[:nocc, :nvir, :nocc, :nvir])##aaaa
                        +1.0*numpy.einsum('iajb,iajb->', rpa_DM_ovov, ovov[:nocc, nvir:, nocc:, :nvir])##abba
                        )
print("\nvlx excitation energy:\n", rpa_excitation_energy)

### Transform the one-particle density matrices from the MO to the AO basis

\begin{equation}
    \gamma_{\mu \nu} = \sum_{pq} C_{\mu p} \gamma_{pq} C_{\nu q}
\end{equation}

In [ ]:
# Transform the one-particle density matrices from MO to AO basis
vlx_dm_oo_in_ao = numpy.matmul(mo_occ, numpy.matmul(vlx_dm_oo, mo_occ.T))
adcc_dm_oo_in_ao = 2.0*numpy.matmul(mo_occ, numpy.matmul(adcc_dm_oo[:nocc,:nocc], mo_occ.T))

vlx_dm_vv_in_ao = numpy.matmul(mo_vir, numpy.matmul(vlx_dm_vv, mo_vir.T))
adcc_dm_vv_in_ao = 2.0*numpy.matmul(mo_vir, numpy.matmul(adcc_dm_vv[:nvir,:nvir], mo_vir.T))

vlx_dm_ao = vlx_dm_oo_in_ao + vlx_dm_vv_in_ao
adcc_dm_ao = adcc_dm_oo_in_ao + adcc_dm_vv_in_ao

print("VLX:")
print(vlx_dm_ao)
print("\nADCC:")
print(adcc_dm_ao)

If everything was transformed correctly, the following equality must hold:
\begin{equation}
    \sum_{pq} \gamma_{pq} \, f_{pq} = \sum_{\mu \nu} \gamma_{\mu \nu} \, f_{\mu \nu}
\end{equation}

In [ ]:
# Confirm the densities by comparing the contractions with the Fock matrix in MO and AO basis

contract_fock_mo = (1.0*numpy.einsum('ij,ij->', vlx_dm_oo, foo[:nocc,:nocc])
                    +1.0*numpy.einsum('ab,ab->', vlx_dm_vv, fvv[:nvir,:nvir]))

contract_fock_ao = numpy.einsum('mn,mn->', vlx_dm_ao, fock)

print("contract_fock_ao =", contract_fock_ao)
print("contract_fock_mo =", contract_fock_mo)

In [ ]:
# Get the total density (including SCF part)
# and calculate the unrelaxed excited-state dipole moment

# dipole integrals
dipole_drv = vlx.ElectricDipoleIntegralsDriver(comm)
dipole_mats = dipole_drv.compute(molecule, basis)
dipole_ints = (dipole_mats.x_to_numpy(),
              dipole_mats.y_to_numpy(),
              dipole_mats.z_to_numpy())

# electronic contribution
total_density = scfdrv.scf_tensors['D'][0] + scfdrv.scf_tensors['D'][1] + vlx_dm_ao
adcc_total_density = scfdrv.scf_tensors['D'][0] + scfdrv.scf_tensors['D'][1] + adcc_dm_ao
electronic_dipole = -1.0 * numpy.array(
    [numpy.sum(dipole_ints[d] * total_density) for d in range(3)])
adcc_electronic_dipole = -1.0 * numpy.array(
    [numpy.sum(dipole_ints[d] * adcc_total_density) for d in range(3)])

# nuclear contribution
coords = molecule.get_coordinates()
nuclear_charges = molecule.elem_ids_to_numpy()
nuclear_dipole = numpy.sum(coords.T * nuclear_charges, axis=1)

dipole_moment = (nuclear_dipole + electronic_dipole)
adcc_dipole_moment = (nuclear_dipole + adcc_electronic_dipole)

print("vlx dipole moment     [au] =", dipole_moment)
print()
print("adcc dm dipole moment [au] =", adcc_dipole_moment)
print()
print("ADC(1) dipole moment from adcc [au]:")
print("                            ", adc1.state_dipole_moment[0])

#### Transform the excitation vectors to the AO basis
\begin{equation}
    x_{\mu \nu} = \sum_{ia} C_{\mu i} x_{ia} C_{\nu a}
\end{equation}

In [ ]:
# Transform the transition density matrix (the excitation vectors) from MO to AO basis
# (will be needed for the two-particle density matrix)

tda_eig_vec_ao = numpy.matmul(mo_occ, numpy.matmul(tda_eig_vec_as_mat, mo_vir.T))
print(tda_eig_vec_ao)

In [ ]:
# Get the ERI tensor in AO basis (chemists' notation)

nao = mo.shape[0]

pqrs = numpy.zeros((nao, nao, nao, nao))
eri_drv = vlx.ElectronRepulsionIntegralsDriver(comm)
eri_drv.compute_in_mem(molecule, basis, pqrs)

The MO to AO transformation of the contribution of the two-particle density matrix with the ERI tensor is given as
\begin{align}
\begin{split}
    - \sum_{\sigma \sigma'} \sum_{ijab} x_{i_{\sigma} b_{\sigma}} x_{j_{\sigma'} a_{\sigma'}}
        \langle i_{\sigma} a_{\sigma'} || j_{\sigma'} b_{\sigma} \rangle
    &= - \sum_{ijab} x_{ib} x_{ja} [ 2 (ij|ab) - 4 (ib|aj) ] \\
    &= + \sum_{ijab} x_{ib} x_{ja} [ 4 (ib|aj) - 2 (ij|ab)  ] \\
    &= \sum_{ijab} x_{ib} x_{ja} \sum_{\mu \nu \theta \varphi}
        C_{\mu i} C_{\nu b} C_{\theta a} C_{\varphi j}  [ 4 (\mu \nu | \theta \varphi)
        - 2 (\mu \varphi | \theta \nu) ] \\
    &= \sum_{\mu \nu \theta \varphi} [ 4 (\mu \nu | \theta \varphi)
        - 2 (\mu \varphi | \theta \nu) ]
        \sum_{ib} C_{\mu i} x_{ib} C_{\nu b} \sum_{ja} C_{\varphi j} x_{ja} C_{\theta a} \\
    &= \sum_{\mu \nu \theta \varphi} x_{\mu \nu} x_{\varphi \theta} [ 4 (\mu \nu | \theta \varphi)
        - 2 (\mu \varphi | \theta \nu) ]       \\
    &= \sum_{\mu \nu} x_{\mu \nu} \sum_{\theta \varphi} x_{\varphi \theta} [ 4 (\mu \nu | \theta \varphi)
        - 2 (\mu \varphi | \theta \nu) ]
\end{split}
\end{align}

In [ ]:
# Confirm by comparing the contraction of electron repulsion integrals in AO basis

# First reference value in MO
eri_contract_mo_adcc = numpy.einsum('iajb,iajb->', adcc_DM_ovov, ovov)
print("Contraction in MO adcc:", eri_contract_mo_adcc)

eri_contract_mo_vlx = (+1.0*numpy.einsum('iajb,iajb->', vlx_DM_ovov, ovov[:nocc, :nvir, :nocc, :nvir])##aaaa
                        +1.0*numpy.einsum('iajb,iajb->', vlx_DM_ovov, ovov[:nocc, nvir:, nocc:, :nvir]))##bbbb
print("Contraction in MO vlx: ", eri_contract_mo_vlx)

# Now contract everything in AO
eri_contract_ao = (0.5*numpy.einsum('mn,mn->', tda_eig_vec_ao, numpy.einsum('pt,mntp->mn', tda_eig_vec_ao, 4*pqrs)
                                   - numpy.einsum('pt,mptn->mn', tda_eig_vec_ao, 2*pqrs))
                    )
print("Contraction in AO:     ", eri_contract_ao)

# Orbital response

\begin{equation}
   -\sum_{pq} \lambda_{pq} \big( [pq|ai] - [pi|aq] \big) + \lambda_{ia} ( \varepsilon_i - \varepsilon_a ) = 
   R_{ia}
   %R^{\text{CIS}}_{ia}
\end{equation}

## Right-hand side

### 1. Contributions from the 1PDMs
\begin{align}
F^\gamma_{ia} &= \sum_{\theta,\varphi}C^\alpha_{\varphi i}C^\alpha_{\theta a}\sum_{\mu,\nu,\sigma}\gamma^\sigma_{\mu\nu}(\mu\nu|\theta \varphi)-\sum_{\theta,\varphi}C^\alpha_{\varphi i}C^\alpha_{\theta a}\sum_{\mu,\nu}\gamma^{\alpha}_{\mu\nu}(\mu\varphi|\theta\nu)\\
&= \sum_{\theta,\varphi}C^\alpha_{\varphi i}C^\alpha_{\theta a} 2F^{1,\alpha}_{\varphi\theta}-\sum_{\theta,\varphi}C^\alpha_{\varphi i}C^\alpha_{\theta a}F^{2,\alpha}_{\varphi\theta}\\
&= \sum_{\theta,\varphi}C^\alpha_{\varphi i}C^\alpha_{\theta a}F_{\varphi\theta}
\end{align}

In [ ]:
# Calculate two different parts and transform the sum to MO

F1_vlx = numpy.einsum('mn,mntp->pt', vlx_dm_ao, pqrs)
F2_vlx = numpy.einsum('mn,mptn->pt', vlx_dm_ao, pqrs)
# The density matrices already have a factor of 2 (for alpρφha and beta spin)
# hence a factor of 0.5 where the other spin part actually vanishes
F_1pdm_vlx = F1_vlx - 0.5*F2_vlx
vlx_1pdm_rhs = numpy.einsum('pi,pa->ia', mo_occ, numpy.einsum('ta,pt->pa', mo_vir, F_1pdm_vlx))
print(vlx_1pdm_rhs)

In [ ]:
# Compare to the analogous calculation with adcc in MO basis

ooov = adc1.reference_state.eri("o1o1o1v1").to_ndarray()
ovvv = adc1.reference_state.eri("o1v1v1v1").to_ndarray()
adcc_1pdm_rhs = ( -numpy.einsum('jk,ikja->ia', adcc_dm_oo, ooov, optimize=True) 
                 -numpy.einsum('bc,icba->ia', adcc_dm_vv, ovvv, optimize=True)
                )
print(adcc_1pdm_rhs)

##### Using AODensityMatrix and LinearSolver objects and comp_lr_fock function

In [ ]:
# Use AODensityMatrix object to compute a Fock-like matrix
# expect something like F_{\varphi\theta}, equation above
# see linearsolver.py; we have to create a linear solver object:

from veloxchem.linearsolver import LinearSolver
from veloxchem.veloxchemlib import AOFockMatrix
from veloxchem.veloxchemlib import AODensityMatrix
from veloxchem.veloxchemlib import denmat

ao_density_object = AODensityMatrix([vlx_dm_ao], denmat.rest)

In [ ]:
linear_solver_object = LinearSolver(comm, ostream)
eri_dict = linear_solver_object.init_eri(molecule, basis)
# DFT information
dft_dict = linear_solver_object.init_dft(molecule, scfdrv.scf_tensors)
# PE information
pe_dict = linear_solver_object.init_pe(molecule, basis)
timing_dict = {}
fock_matrix_rhs_1pdm = AOFockMatrix(ao_density_object)
#print(fock_matrix_rhs_1pdm)
linear_solver_object.comp_lr_fock(fock_matrix_rhs_1pdm, ao_density_object, molecule, basis,
                                  eri_dict, dft_dict, pe_dict, timing_dict)
print("Fock matrix from 1PDM, from LinearSolver:")
print(fock_matrix_rhs_1pdm)
print()
print("Fock matrix from 1PDM, using numpy.einsum:")
print(2*F_1pdm_vlx)
#We have to take 0.5*fock_matrix_rhs_1pdm because there is a sqrt(2) in the excitation vectors
#(normalized? alpha+beta?, but we are looking only at the alpha-alpha block of the multipliers) 
rhs_1pdm_from_LinearSolver = numpy.einsum('pi,pa->ia', mo_occ,
                            numpy.einsum('ta,pt->pa', mo_vir, 0.5*fock_matrix_rhs_1pdm.alpha_to_numpy(0)))
print("\nIn MO basis, 1PDM contribution to the RHS, LinearSolver:")
print(rhs_1pdm_from_LinearSolver)
print()
print("Original, for comparison:")
print(vlx_1pdm_rhs)

### 2. Contributions from the 2PDMs

\begin{align}
F^\Gamma_{ia} &= ~~ \sum_{\mu,\zeta}C^\alpha_{\mu i}C^\alpha_{\zeta a} \Big[\sum_{\rho,\varphi}S_{\rho\zeta}x^\alpha_{\varphi\rho}\sum_{\nu,\theta}x^\alpha_{\theta\nu}(\mu\nu|\theta\varphi)-\sum_{\rho,\varphi}S_{\rho\zeta}x^\alpha_{\varphi\rho}\sum_{\theta,\nu,\sigma} x^\sigma_{\theta\nu}(\mu\varphi|\theta\nu) \Big] \\
&~~~ -\sum_{\mu,\zeta}C^\alpha_{\mu i}C^\alpha_{\zeta a} \Big[ \sum_{\rho,\varphi}S_{\mu\rho}x^\alpha_{\rho\varphi}\sum_{\nu,\theta}x^\alpha_{\nu\theta}(\zeta\nu|\theta\varphi)-\sum_{\rho,\varphi} S_{\mu\rho}x^\alpha_{\rho\varphi}\sum_{\theta,\nu,\sigma}x^\sigma_{\nu\theta}(\zeta\varphi|\theta\nu) \Big] \\
&= - \sum_{\mu,\zeta}C^\alpha_{\mu i}C^\alpha_{\zeta a} \sum_{\rho,\varphi}S_{\rho\zeta}x^\alpha_{\varphi\rho}\Big[ 2\sum_{\theta,\nu} x^\alpha_{\theta\nu}(\mu\varphi|\theta\nu)- \sum_{\nu,\theta}x^\alpha_{\theta\nu}(\mu\nu|\theta\varphi) \Big] \\
&~~~~ +\sum_{\mu,\zeta}C^\alpha_{\mu i}C^\alpha_{\zeta a} \sum_{\rho,\varphi}S_{\mu\rho}x^\alpha_{\rho\varphi}\Big[ 2\sum_{\theta,\nu} x^\alpha_{\nu\theta}(\zeta\varphi|\theta\nu)- \sum_{\nu,\theta}x^\alpha_{\nu\theta}(\zeta\nu|\theta\varphi) \Big] \\
&= - \sum_{\mu,\zeta}C^\alpha_{\mu i}C^\alpha_{\zeta a} \sum_{\rho,\varphi}S_{\rho\zeta}x^\alpha_{\varphi\rho}\Big[ \sum_{\theta,\nu} x^\alpha_{\theta\nu} \big[ 2 (\mu\varphi|\theta\nu)- (\mu\nu|\theta\varphi) \big] \Big] \\
&~~~~ +\sum_{\mu,\zeta}C^\alpha_{\mu i}C^\alpha_{\zeta a} \sum_{\rho,\varphi}S_{\mu\rho}x^\alpha_{\rho\varphi}\Big[ \sum_{\theta,\nu} x^\alpha_{\nu\theta} \big[ 2(\zeta\varphi|\theta\nu)- (\zeta\nu|\theta\varphi) \big] \Big]
\end{align}

In [ ]:
# Care needs to be taken with the index order of the excitation vectors
# Calculate in different summation orders...

# 1)
FG1 = ( numpy.einsum('rz,mr->mz', ovlp,
                    numpy.einsum('pr,mp->mr', tda_eig_vec_ao,
                        (0.5*numpy.einsum('tn,mntp->mp',tda_eig_vec_ao,pqrs)
                        -numpy.einsum('tn,mptn->mp',tda_eig_vec_ao,pqrs))
                                )
                      )
            )
FG2 = ( numpy.einsum('mr,rz->mz', ovlp,
                    numpy.einsum('rp,zp->rz', tda_eig_vec_ao,
                        (0.5*numpy.einsum('nt,zntp->zp',tda_eig_vec_ao,pqrs)
                        -numpy.einsum('nt,zptn->zp',tda_eig_vec_ao,pqrs))
                                )
                      )
            )

# 2)
FG1_try2 =( numpy.einsum('rz,mr->mz', ovlp,
                       numpy.einsum('pr,mp->mr', tda_eig_vec_ao,
                                    numpy.einsum('tn,mntp->mp',tda_eig_vec_ao,pqrs)
                                   )
                      )
            )
FG2_try2 =( numpy.einsum('rz,mr->mz', ovlp,
                       numpy.einsum('pr,mp->mr', tda_eig_vec_ao,
                                    numpy.einsum('tn,mptn->mp',tda_eig_vec_ao,pqrs)
                                   )
                      )
            )
FG3_try2 =( numpy.einsum('mr,rz->mz', ovlp,
                       numpy.einsum('rp,zp->rz', tda_eig_vec_ao,
                                    numpy.einsum('nt,zntp->zp',tda_eig_vec_ao,pqrs)
                                   )
                      )
            )
FG4_try2 =( numpy.einsum('mr,rz->mz', ovlp,
                       numpy.einsum('rp,zp->rz', tda_eig_vec_ao,
                                    numpy.einsum('nt,zptn->zp',tda_eig_vec_ao,pqrs)
                                   )
                      )
            )

FG1_vlx =( numpy.einsum('rz,zm->rm', ovlp,
                       numpy.einsum('zp,mp->zm', tda_eig_vec_ao,
                                    numpy.einsum('tn,mntp->mp',tda_eig_vec_ao,pqrs)
                                   )
                      )
            )
FG1_vlx_ovlp =( numpy.einsum('rp,mp->rm',
                numpy.einsum('rz,zp->rp', ovlp, tda_eig_vec_ao),
                numpy.einsum('tn,mntp->mp',tda_eig_vec_ao,pqrs)
                                   )
            )

FG2_vlx =  numpy.einsum('rz,zm->rm', ovlp,
                       numpy.einsum('zn,nm->zm', tda_eig_vec_ao,
                                    numpy.einsum('tp,mntp->nm',tda_eig_vec_ao,pqrs)
                                   )
                      )

# Sum up terms accounting for spin factors
F_2pdm_vlx_A = 0.5*FG1_try2 - FG2_try2 
F_2pdm_vlx_B =-0.5*FG3_try2 + FG4_try2

F_2pdm_vlx = F_2pdm_vlx_A + F_2pdm_vlx_B
#print(F_2pdm_vlx)
# 3)
another_vlx_2pdm_rhs = (numpy.einsum('mi,ma->ia', mo_occ, numpy.einsum('za,mz->ma', mo_vir, F_2pdm_vlx_A))
                +numpy.einsum('mi,ma->ia', mo_occ, numpy.einsum('za,mz->ma', mo_vir, F_2pdm_vlx_B))
               )
vlx_2pdm_rhs = (numpy.einsum('mi,ma->ia', mo_occ, numpy.einsum('za,mz->ma', mo_vir, F_2pdm_vlx)))
final_vlx_2pdm_rhs = (numpy.einsum('mi,ma->ia', mo_occ, numpy.einsum('za,mz->ma', mo_vir, FG1-FG2)))
print(vlx_2pdm_rhs)
print()
print(another_vlx_2pdm_rhs)         
print()
print(final_vlx_2pdm_rhs)

In [ ]:
# Compare to adcc
adcc_2pdm_rhs = (numpy.einsum('jakb,ijkb->ia', adcc_DM_ovov, ooov, optimize=True)
                + numpy.einsum('ibjc,jcab->ia', adcc_DM_ovov, ovvv, optimize=True)
                )
print(adcc_2pdm_rhs)

In [ ]:
# 2PDM contribution, using AODensityMatrix, AOFockMatrix, LinearSolver
# we need the transpose of the vector because of how the exchange part is calculated
# (mn|tp)!=(mt|np), but (mn|tp)=(nm|tp)=(mn|pt)=(nm|pt) real orbitals

ao_density_2pdm_1 = AODensityMatrix([tda_eig_vec_ao.T], denmat.rest)
ao_density_2pdm_2 = AODensityMatrix([tda_eig_vec_ao], denmat.rest)
ao_fock_2pdm_1 = AOFockMatrix(ao_density_2pdm_1)
ao_fock_2pdm_2 =AOFockMatrix(ao_density_2pdm_2)
linear_solver_2pdm_1 = LinearSolver(comm, ostream) #Can we use the same object?
linear_solver_2pdm_1.comp_lr_fock(ao_fock_2pdm_1, ao_density_2pdm_1, molecule, basis,
                                  eri_dict, dft_dict, pe_dict, timing_dict)
linear_solver_2pdm_2 = LinearSolver(comm, ostream) #Can we use the same object?
linear_solver_2pdm_2.comp_lr_fock(ao_fock_2pdm_2, ao_density_2pdm_2, molecule, basis,
                                  eri_dict, dft_dict, pe_dict, timing_dict)
                        
fock_1 = (1.0*numpy.einsum('tn,mntp->mp',tda_eig_vec_ao,pqrs)
        -2.0*numpy.einsum('mn,mnpt->pt',tda_eig_vec_ao,pqrs))

fock_2 = (1.0*numpy.einsum('nt,zntp->zp',tda_eig_vec_ao,pqrs)
        -2.0*numpy.einsum('nt,zptn->zp',tda_eig_vec_ao,pqrs))

print("What we are after, 1st term:")
print(fock_1)
print()
print("What we get from comp_lr_fock using tda_eig_vec_ao.T:")
print(ao_fock_2pdm_1)
print()
print("What we are after, 2nd term:")
print(fock_2)
print()
print("What we get from comp_lr_fock using tda_eig_vec_ao:")
print(ao_fock_2pdm_2)
print()
LR_2pdm_F1 =  numpy.einsum('rz,mr->mz', ovlp,
             numpy.einsum('pr,mp->mr', tda_eig_vec_ao, 0.5*ao_fock_2pdm_1.alpha_to_numpy(0)))
LR_2pdm_F2 =  numpy.einsum('mr,rz->mz', ovlp,
                    numpy.einsum('rp,zp->rz', tda_eig_vec_ao, 0.5*ao_fock_2pdm_2.alpha_to_numpy(0)))

rhs_2pdm_from_LinearSolver = numpy.einsum('mi,ma->ia', mo_occ, 
                                          numpy.einsum('za,mz->ma', mo_vir, LR_2pdm_F2-LR_2pdm_F1))
print("2PDM RHS using LinearSolver:")
print(rhs_2pdm_from_LinearSolver)
print()
print("2PDM RHS using numpy.einsum:")
print(final_vlx_2pdm_rhs)

## Left-hand side
### Initial guess for Lagrange multipliers $\boldsymbol{\lambda}$

\begin{equation}
    \lambda_{ia}^{(0)} = \frac{R_{ia}}{\varepsilon_i - \varepsilon_a}
\end{equation}

In [ ]:
# Neglect the non-diagonal part of the LHS,
# i.e. divide the RHS by the orbital-energy difference

vlx_rhs = final_vlx_2pdm_rhs + vlx_1pdm_rhs
lambda_guess = numpy.zeros((nocc,nvir))
mo_energies = scfdrv.scf_tensors['E']
#print(mo_energies[nocc:])

for i in range(nocc):
    ei=mo_energies[i]
    for a in range(nvir):
        ea=mo_energies[nocc+a]
        lambda_guess[i,a] = vlx_rhs[i,a]/(ei-ea)
        
print(lambda_guess)

In [ ]:
# Compare to adcc
foo = adc1.reference_state.fock("o1o1").to_ndarray() 
fvv = adc1.reference_state.fock("v1v1").to_ndarray()

adcc_rhs = adcc_1pdm_rhs + adcc_2pdm_rhs
adcc_guess = adcc_rhs.copy() # rhs
for i in range(foo.shape[0]):
    for a in range(fvv.shape[0]):
        adcc_guess[i,a] = adcc_guess[i,a]/(foo[i,i]-fvv[a,a])

print(adcc_guess)

In [ ]:
# Transform the initial guess for lambda to the AO basis

lambda_guess_ao = numpy.matmul(mo_occ, numpy.matmul(lambda_guess, mo_vir.T))
print(lambda_guess_ao)

# Transform back to MO basis to check
#lambda_guess_back_to_mo = numpy.matmul(mo_occ.T, numpy.matmul(ovlp, numpy.matmul(lambda_guess_ao, numpy.matmul(ovlp, mo_vir))))
#print(lambda_guess_back_to_mo)

### Contraction of the initial guess for $\boldsymbol{\lambda}$ with two-electron integrals

using $p=j, q=b$ as well as $p=b, q=j$ with the condition $\lambda_{pq} = \lambda_{qp}$ and assuming real orbitals, the equation is obtained as
\begin{align}
-\sum_{pq} \lambda_{pq} \big( [pq|ai] - [pi|aq] \big)
    &= -\sum_{\varphi \theta} C_{\varphi i} C_{\theta a} \sum_{\mu \nu}
        \lambda_{\mu \nu} \big[ 4 (\mu \nu | \varphi \theta)
        - (\mu \theta | \varphi \nu) - (\nu \theta | \varphi \mu) \big] \\
    %+ (\varepsilon_a - \varepsilon_i ) \gamma_{ia}
    &= - \sum_{\varphi \theta} \big[ C_{\varphi i} C_{\theta a} + C_{\theta i} C_{\varphi a} \big]
        \sum_{\mu \nu} \lambda_{\mu \nu} \big[ 2 (\mu \nu | \varphi \theta) - (\mu \theta | \varphi \nu) \big]
\end{align}

In [ ]:
# Contract with the ERI tensor
vlx_lhs1 = numpy.einsum('mn,mnpt->pt', lambda_guess_ao, pqrs)
vlx_lhs2 = numpy.einsum('mn,mtpn->pt', lambda_guess_ao, pqrs)
vlx_lhs3 = numpy.einsum('mn,ntpm->pt', lambda_guess_ao, pqrs)

vlx_lhs_ao = -4*vlx_lhs1 + vlx_lhs2 + vlx_lhs3

# Transform to MO
vlx_lhs_mo = numpy.matmul(mo_occ.T, numpy.matmul(vlx_lhs_ao, mo_vir))
#vlx_lhs_mo2 = numpy.einsum('pi,pa->ia', mo_occ, numpy.einsum('ta,pt->pa', mo_vir, vlx_lhs_ao))
print(vlx_lhs_mo)
#print(vlx_lhs_mo2)

In [ ]:
# Another try with different index convention
vlx_lhs5 = numpy.einsum('mn,mnpt->pt', lambda_guess_ao, pqrs)
vlx_lhs6 = numpy.einsum('mn,mtpn->pt', lambda_guess_ao, pqrs)
vlx_lhs7 = numpy.einsum('mn,ntpm->pt', lambda_guess_ao, pqrs)
vlx_lhs_ao_try3 = -4*vlx_lhs5 + vlx_lhs6 + vlx_lhs7

vlx_1pdm_lhs_new = numpy.einsum('pi,pa->ia', mo_occ, numpy.einsum('ta,pt->pa', mo_vir, vlx_lhs_ao_try3))
print(vlx_1pdm_lhs_new)

In [ ]:
#Lambda guess contracted with eri using AODensityMatrix, AOFockMatrix, LinearSolver

ao_density_lambda = AODensityMatrix([lambda_guess_ao], denmat.rest)
ao_fock_lambda = AOFockMatrix(ao_density_lambda)
linear_solver_lambda = LinearSolver(comm, ostream) #Can we use the same object?
linear_solver_lambda.comp_lr_fock(ao_fock_lambda, ao_density_lambda, molecule, basis,
                                  eri_dict, dft_dict, pe_dict, timing_dict)

L1_vlx = numpy.einsum('mn,mnpt->pt', lambda_guess_ao, pqrs)
L2_vlx = numpy.einsum('mn,mptn->pt', lambda_guess_ao, pqrs)
L_total_vlx = 2*L1_vlx - L2_vlx
print("Lambda in AO using AOFockMatrix:")
print(ao_fock_lambda)
print()
print("Lambda in AO using numpy.einsum:")
print(L_total_vlx)
fock_lambda_numpy = ao_fock_lambda.alpha_to_numpy(0)
#print()
#print(fock_lambda_numpy)
lambda_mo_from_fock = ( numpy.matmul(mo_occ.T, numpy.matmul(fock_lambda_numpy, mo_vir)) 
                       + numpy.matmul(mo_vir.T, numpy.matmul(fock_lambda_numpy, mo_occ)).T )
print()
print("In MO, using the Fock Matrix object")
print(lambda_mo_from_fock)
print()
print("In MO, from the Equation:")
print(vlx_lhs_mo)

In [ ]:
# LHS and guess
print("vlx_lhs_mo:")
print(vlx_lhs_mo)
print()
print("lambda_guess:")
print(lambda_guess)

#### Full LHS 
\begin{equation}
   -\sum_{pq} \lambda_{pq} \big( [pq|ai] - [pi|aq] \big) + \lambda_{ia} ( \varepsilon_i - \varepsilon_a )
\end{equation}

In [ ]:
# Add product of guess and orbital-energy differences to get full LHS

vlx_full_lhs = numpy.zeros((nocc,nvir))
for i in range(nocc):
    ei = mo_energies[i]
    for a in range(nvir):
        ea = mo_energies[nocc+a]
        vlx_full_lhs[i,a] = vlx_lhs_mo[i,a] + lambda_guess[i,a]*(ei-ea)

vlx_full_lhs_vec = vlx_full_lhs.reshape(nocc*nvir)
vlx_rhs_vec = vlx_rhs.reshape(nocc*nvir)

print(vlx_full_lhs_vec)
print()
print(vlx_rhs_vec)

## Solution via Conjugate Gradient

To solve: $\mathbf{A} \mathbf{x} = \mathbf{b}$

However, we do not "know" the matrix $\mathbf{A}$, only the matrix-vector product $\mathbf{Ax}$.
Thus, define it as a linear operator with a vector $\mathbf{x}$ as an argument.

### Defining a linear operator

In [ ]:
# Assuming that all other variables (nocc, nvir, MOs, ERIs) are known from the "global" scope
def Ax(v):
    """Function to carry out matrix multiplication
    of Lagrange multipier vector with orbital Hessian
    matrix. Transforms guess to AO basis, carries out
    matrix-vector product and transforms back to MO.
    
    v: lambda at current iteration"""

    current_lambda = v.reshape(nocc,nvir)
    print("VLX INPUT:")
    print(current_lambda)
    print()
    
    # Transform to AO
    vlx_lambda_ao = numpy.matmul(mo_occ, numpy.matmul(current_lambda, mo_vir.T))
 
    # Calculate the different contractions
    vlx_lhs_interm = (-4*numpy.einsum('mn,mntp->pt', vlx_lambda_ao, pqrs)
                    + 1.0*numpy.einsum('mn,mtpn->pt', vlx_lambda_ao, pqrs)
                    + 1.0*numpy.einsum('mn,ntpm->pt', vlx_lambda_ao, pqrs)
                      )
    # Transform back to MO
    vlx_1pdm_lhs = numpy.einsum('pi,pa->ia', mo_occ, numpy.einsum('ta,pt->pa', mo_vir, vlx_lhs_interm))

# Add the diagonal part
    for i in range(nocc):
        ei = mo_energies[i]
        for a in range(nvir):
            ea = mo_energies[nocc+a]
            vlx_1pdm_lhs[i,a] += current_lambda[i,a]*(ei-ea) 
    print("VLX OUTPUT:")    
    print(vlx_1pdm_lhs)
    print()
    return vlx_1pdm_lhs.reshape(nocc*nvir)

In [ ]:
def Ax_with_LinearSolver(v):
    """Function to carry out matrix multiplication
    of Lagrange multipier vector with orbital Hessian
    matrix, using AODensityMatrix, AOFockMatrix, and comp_lr_fock
    Transforms guess to AO basis, carries out
    matrix-vector product and transforms back to MO.
    
    v: lambda at current iteration"""
    
    current_lambda = v.reshape(nocc,nvir)
    #print("LinearSolver VLX INPUT:")
    #print(current_lambda)
    #print()

    # Transform to AO
    vlx_lambda_ao = numpy.matmul(mo_occ, numpy.matmul(current_lambda, mo_vir.T))
    
    #Create AODensityMatrix object from lambda in AO
    vlx_ao_density_lambda = AODensityMatrix([vlx_lambda_ao], denmat.rest)
    
    #Create a Fock Matrix Object (initialized with zeros)
    vlx_fock_lambda = AOFockMatrix(vlx_ao_density_lambda)
    
    #Create linear solver object and compute the Fock matrix (contracts AODensityMatrix with ERI: 2J-K)
    linear_solver_lambda = LinearSolver(comm, ostream) #Can we use the same object?
    linear_solver_lambda.comp_lr_fock(vlx_fock_lambda, vlx_ao_density_lambda, molecule, basis,
                                      eri_dict, dft_dict, pe_dict, timing_dict)

    #Transform AOFockMatrix to numpy array (here we took only the alpha block)
    vlx_fock_lambda_numpy = vlx_fock_lambda.alpha_to_numpy(0)
    
    
    #Transform to MO basis (symmetrized w.r.t. occ. and virt.)
    vlx_lambda_mo_from_fock = -( numpy.matmul(mo_occ.T, numpy.matmul(vlx_fock_lambda_numpy, mo_vir)) 
                               + numpy.matmul(mo_vir.T, numpy.matmul(vlx_fock_lambda_numpy, mo_occ)).T )

    
    # Add the diagonal part
    for i in range(nocc):
        ei = mo_energies[i]
        for a in range(nvir):
            ea = mo_energies[nocc+a]
            vlx_lambda_mo_from_fock[i,a] += current_lambda[i,a]*(ei-ea) 
    #print("LinearSolver VLX OUTPUT:")    
    #print(vlx_lambda_mo_from_fock)
    #print()
    return vlx_lambda_mo_from_fock.reshape(nocc*nvir)

In [ ]:
# Solve via CG

vlx_guess_vec=lambda_guess.reshape(nocc*nvir)

A = linalg.LinearOperator((nocc*nvir,nocc*nvir), matvec=Ax)
solution, w = linalg.cg(A=A, b=vlx_rhs_vec, x0=vlx_guess_vec, tol=1e-8, maxiter=25)

In [ ]:
#Use Ax_with_LinearSolver for conjugate gradient
lr_vlx_guess_vec=lambda_guess.reshape(nocc*nvir)

#print("Run Ax_with_LinearSolver once, lambda_guess as input:")
#Ax_with_LinearSolver(lambda_guess.reshape(nocc*nvir))
#print()

#print("Create scipy.linalg LinearOperator:")
LR_A = linalg.LinearOperator((nocc*nvir,nocc*nvir), matvec=Ax_with_LinearSolver)

LR_full_RHS_1d_vec = (rhs_2pdm_from_LinearSolver + rhs_1pdm_from_LinearSolver).reshape(nocc*nvir)

print("Starting conjugate gradient...\n")
LR_solution, lr_w = linalg.cg(A=LR_A, b=LR_full_RHS_1d_vec, x0=LR_full_RHS_1d_vec.copy(), tol=1e-8, maxiter=25)

print("The solution in matrix form:")
print(LR_solution.reshape(nocc,nvir))
print()
print("The solution using numpy.einsum:")
print(solution.reshape(nocc,nvir))

In [ ]:
# Define the A matrix for adcc

AOVOV = numpy.ones(ovov.shape)   
adcc_nocc = foo.shape[0]
adcc_nvirt = fvv.shape[0]
oovv = adc1.reference_state.eri("o1o1v1v1").to_ndarray()

#these are supposed to be the elements that multiply lambda_jb
AOVOV = ( numpy.einsum('iajb,ibja->iajb', AOVOV, ovov, optimize=True)
         -numpy.einsum('iajb,ijab->iajb', AOVOV, oovv, optimize=True) 
        )

for i in range(adcc_nocc):
    for a in range(adcc_nvirt):
        AOVOV[i,a,i,a] += foo[i,i]-fvv[a,a] #check signs!

adcc_A = AOVOV.reshape(adcc_nocc*adcc_nvirt,adcc_nocc*adcc_nvirt)

In [ ]:
# Solve regularly via conjugate gradient from scipy.sparse

vlx_guess_vec=lambda_guess.reshape(nocc*nvir)

adcc_1d_guess = numpy.reshape(adcc_guess, (adcc_nocc*adcc_nvirt))
adcc_1d_rhs = numpy.reshape(adcc_rhs, (adcc_nocc*adcc_nvirt))

lov, w = linalg.cg(A=adcc_A, b=adcc_1d_rhs, x0=adcc_1d_guess, tol=1e-8, maxiter=25)

In [ ]:
# Define a linear operator also for adcc in the MO basis

def adcc_Ax_MO(v):
    print("Entering Ax_MO(v)...:")
    new_v = v.reshape(adcc_nocc,adcc_nvirt)
    print("ADCC INPUT")
    print(new_v)
    print()
    adcc_1pdm_lhs = ( numpy.einsum('kb,kaib->ia', new_v, ovov, optimize=True)
         -numpy.einsum('kb,kiba->ia', new_v, oovv, optimize=True) 
        )

    # Add the diagonal part
    for i in range(adcc_nocc):
        ei = foo[i,i]
        for a in range(adcc_nvirt):
            ea = fvv[a,a]
            adcc_1pdm_lhs[i,a] += new_v[i,a]*(ei-ea)
    print("ADCC OUTPUT")
    print(adcc_1pdm_lhs)
    print()
    return adcc_1pdm_lhs.reshape(adcc_nocc*adcc_nvirt)

# Solve via CG
adcc_Ax = linalg.LinearOperator((adcc_nocc*adcc_nvirt,adcc_nocc*adcc_nvirt), matvec=adcc_Ax_MO)
adcc_solution, adcc_w = linalg.cg(A=adcc_Ax, b=adcc_1d_rhs, x0=adcc_1d_guess, tol=1e-8, maxiter=25)

In [ ]:
adcc_1d_guess = numpy.reshape(adcc_guess, (adcc_nocc*adcc_nvirt))
adcc_1d_rhs = numpy.reshape(adcc_rhs, (adcc_nocc*adcc_nvirt))

lov, w = linalg.cg(A=adcc_A, b=adcc_1d_rhs, x0=adcc_1d_guess, tol=1e-8, maxiter=25)

In [ ]:
print("MO basis, ADCC solution, regular CG")
print(lov.reshape(adcc_nocc,adcc_nvirt))
print()
print("MO basis, ADCC solution, linear operator")
print(adcc_solution.reshape(adcc_nocc, adcc_nvirt))
print()
print("AO basis, VLX solution, linear operator")
print(solution.reshape(nocc,nvir))